In [19]:
import numpy as np
import trimesh
import scipy

In [1]:
# create test and train set 
# save it as txt files
import glob
import random
def get_random_train_list():
    files = glob.glob('data/noisy_clouds/*.xyz')
    train = random.sample(files, 120)
    train_set = set(train)
    train_list = []
    test_list = []
    for file in files:
        name = file.split('/')[-1].split('.')[0]
        if file in train_set:
            train_list.append(name)
        else:
            test_list.append(name)

    return train_list, test_list
def write_list_to_file(filename,my_list):
    with open(filename, 'w') as f:
        for item in my_list:
            f.write("%s\n" % item)
train_list, test_list = get_random_train_list()
write_list_to_file('data/noisy_clouds/trainingset.txt', train_list)
write_list_to_file('data/noisy_clouds/testset.txt', test_list)

In [144]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import coo_matrix, eye


from trimesh import triangles


def filter_humphrey(mesh,
                    alpha=0.1,
                    beta=0.5,
                    iterations=10,
                    laplacian_operator=None):
    """
    Smooth a mesh in-place using laplacian smoothing
    and Humphrey filtering.
    Articles
    "Improved Laplacian Smoothing of Noisy Surface Meshes"
    J. Vollmer, R. Mencl, and H. Muller
    Parameters
    ------------
    mesh : trimesh.Trimesh
      Mesh to be smoothed in place
    alpha : float
      Controls shrinkage, range is 0.0 - 1.0
      If 0.0, not considered
      If 1.0, no smoothing
    beta : float
      Controls how aggressive smoothing is
      If 0.0, no smoothing
      If 1.0, full aggressiveness
    iterations : int
      Number of passes to run filter
    laplacian_operator : None or scipy.sparse.coo.coo_matrix
      Sparse matrix laplacian operator
      Will be autogenerated if None
    """
    # if the laplacian operator was not passed create it here
    if laplacian_operator is None:
        laplacian_operator = laplacian_calculation(mesh)

    # get mesh vertices as vanilla numpy array
    vertices = mesh.vertices.copy().view(np.ndarray)
    # save original unmodified vertices
    original = vertices.copy()

    # run through iterations of filter
    for _index in range(iterations):
        vert_q = vertices.copy()
        vertices = laplacian_operator.dot(vertices)
        vert_b = vertices - (alpha * original + (1.0 - alpha) * vert_q)
        vertices -= (beta * vert_b + (1.0 - beta) *
                     laplacian_operator.dot(vert_b))

    # assign modified vertices back to mesh
    mesh.vertices = vertices
    return mesh, laplacian_operator
def laplacian_calculation(mesh, equal_weight=True):
    """
    Calculate a sparse matrix for laplacian operations.
    Parameters
    -------------
    mesh : trimesh.Trimesh
      Input geometry
    equal_weight : bool
      If True, all neighbors will be considered equally
      If False, all neightbors will be weighted by inverse distance
    Returns
    ----------
    laplacian : scipy.sparse.coo.coo_matrix
      Laplacian operator
    """
    # get the vertex neighbors from the cache
    neighbors = mesh.vertex_neighbors
    print(neighbors)
    # avoid hitting crc checks in loops
    vertices = mesh.vertices.view(np.ndarray)
    print('vertices: \n', vertices)

    # stack neighbors to 1D arrays
    col = np.concatenate(neighbors)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(neighbors)])

    if equal_weight:
        # equal weights for each neighbor
        data = np.concatenate([[1.0 / len(n)] * len(n)
                               for n in neighbors])
    else:
        # umbrella weights, distance-weighted
        # use dot product of ones to replace array.sum(axis=1)
        ones = np.ones(3)
        # the distance from verticesex to neighbors
        norms = [1.0 / np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                 for i, n in enumerate(neighbors)]
        # normalize group and stack into single array
        data = np.concatenate([i / i.sum() for i in norms])

    # create the sparse matrix
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)

    return matrix
def laplacian_cotagent(mesh):
  
    # get the vertex neighbors from the cache
    neighbors = mesh.vertex_neighbors

    # avoid hitting crc checks in loops
    vertices = mesh.vertices.view(np.ndarray)

    # stack neighbors to 1D arrays
    col = np.concatenate(neighbors)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(neighbors)])

    if equal_weight:
        # equal weights for each neighbor
        data = np.concatenate([[1.0 / len(n)] * len(n)
                               for n in neighbors])
    else:
        # umbrella weights, distance-weighted
        # use dot product of ones to replace array.sum(axis=1)
        ones = np.ones(3)
        # the distance from verticesex to neighbors
        norms = [1.0 / np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                 for i, n in enumerate(neighbors)]
        # normalize group and stack into single array
        data = np.concatenate([i / i.sum() for i in norms])

    # create the sparse matrix
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)

    return matrix
mesh = trimesh.load_mesh('../non_manifold_plane.obj')
# matrix = laplacian_calculation(mesh)

In [134]:
neighbors = mesh.vertex_neighbors
col = np.concatenate(neighbors)
col

array([1, 2, 6, 7, 0, 7, 0, 8, 6, 8, 4, 5, 6, 3, 6, 7, 8, 3, 0, 2, 3, 4,
       7, 8, 0, 1, 4, 6, 2, 3, 5, 6])

In [138]:
row = np.concatenate([[i] * len(n) for i, n in enumerate(neighbors)])
row

array([0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6,
       6, 6, 7, 7, 7, 7, 8, 8, 8, 8])

In [139]:
data = np.concatenate([[1.0 / len(n)] * len(n) for n in neighbors])
data

array([0.25      , 0.25      , 0.25      , 0.25      , 0.5       ,
       0.5       , 0.33333333, 0.33333333, 0.33333333, 0.25      ,
       0.25      , 0.25      , 0.25      , 0.33333333, 0.33333333,
       0.33333333, 0.5       , 0.5       , 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.25      ,
       0.25      , 0.25      , 0.25      , 0.25      , 0.25      ,
       0.25      , 0.25      ])

In [143]:
# [len(mesh.vertices)] * 2  === 9 * 9 matrix
matrix = coo_matrix((data, (row, col)), shape=[len(mesh.vertices)] * 2)
print(matrix.shape)


(9, 9)


In [86]:
mesh = trimesh.load_mesh('../noisy_mesh/meshes_bouncing_mesh_0020.obj')
matrix = laplacian_calculation(mesh)
# np.save('a.npy', matrix)


smoothed_mesh = trimesh.smoothing.filter_taubin(mesh,
                  lamb=0.5,
                  nu=0.5,
                  iterations=10,
                  laplacian_operator=None)
smoothed_mesh.show()

In [18]:
import torch
import numpy as np
import scipy.sparse
center_point_ind = 6
laplacian_filename = 'data/laplacian/meshes_bouncing_mesh_0000.npz'
laplacian = scipy.sparse.load_npz(laplacian_filename)
laplacian = laplacian.toarray()
print(laplacian.shape)
# patch_laplacian = torch.from_numpy(laplacian[center_point_ind, :]) 

(10002, 10002)


In [87]:
a = smoothed_mesh.export('../filter_taubin.off', 'off')

In [96]:
import igl
import meshplot as mp

v,f = igl.read_triangle_mesh( '../meshes_bouncing_mesh_0020.obj')
v_pnt = np.loadtxt('../pointcleannet_meshes_bouncing_mesh_0020.xyz')
p = mp.plot(v_pnt,f)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.014683…

In [94]:
p.save('../ground_truth.html')

Plot saved to file ../ground_truth.html.


In [167]:
# Replace the Trimesh Laplacian with the cotangent Laplacian
# Have a function that computes the cotangent weights, given a center point and its k neighbors
# Compute k nearest neighbors
# For each of these k nearest neighbors,
# check if there is an edge in the between the center point and this neighbor. If not, return 0
# If yes, return cotagent weight

# Aijk = np.sqrt(s(s−lik)(s−lkj)(s−lij))
# s = (lik +lkj +lij)/2

# wij = lij + lik + ljk/(8*Aijk) + lij + lih + ljh/(8*Aijh)
mesh = trimesh.load_mesh('../2plane.obj')
vertices = mesh.vertices

# def get_k_nearest_neighbors(vertices):
import heapq
import numpy as np
get_distance = lambda p1, p2: np.sqrt(np.sum((p1 - p2) ** 2, axis=0))

result = []
for i,p1 in enumerate(vertices):
    # print('=================')
    # print('        '+ str(i))
    # print('=================')
    h = []
    for j, p2 in enumerate(vertices):
        if i == j:
            continue
        dist = (get_distance(p1,p2))
        heapq.heappush(h,(dist, j))
    result.append(h)
print(result[1])
        
    



    


[(TrackedArray(1.), 0), (TrackedArray(2.), 2), (TrackedArray(2.23606798), 3), (TrackedArray(2.), 4), (TrackedArray(2.82842712), 5)]


In [166]:
# mesh.edges
heapq.heappop(result[1])

(TrackedArray(2.), 4)

In [156]:
mesh.vertex_neighbors

[[1, 2, 3, 4], [0, 4], [0, 3, 5], [0, 2, 4, 5], [0, 1, 3], [2, 3]]